# forward model

In [1]:
import numpy as np
import h5py

In [2]:
params = {
    'model_weight' : '../ner_model_weight/model_conv_625.h5',
    'embed_size' : 500,
    'max_sent_len': 20
}

In [3]:
import json
with open('../char_6.17.json', mode='r', encoding='utf-8') as f:
    dicts = json.load(f)
char2id = dicts['char2id']
id2char = dicts['id2char']
intent2id = dicts['intent2id']
id2intent = dicts['id2intent']
slot2id = dicts['slot2id']
id2slot = dicts['id2slot']

In [4]:
def sigmoid(x):
    y = 1 / (1 + np.exp(-x))
    return y

In [5]:
def tanh(x):
    y=(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    return y

In [6]:
def embedding(x,embed_size,embed):
    x_one= np.zeros((len(x),embed_size))
    x_one[range(len(x)), x] = 1
    x_embed = np.dot(x_one, embed)
    return x_embed

In [10]:
def GRU(cell_inputs,cell_states,kernel,recurrent_kernel,bias):
    input_bias = bias[0]
    recurrent_bias = bias[1]
    
    matrix_x = np.dot(cell_inputs, kernel)
    matrix_x = np.add(matrix_x, input_bias)
    
    x_z, x_r, x_h = np.split(matrix_x,3, axis=-1)
    
    matrix_inner = np.dot(cell_states, recurrent_kernel)
    matrix_inner = np.add(matrix_inner, recurrent_bias)
    
    recurrent_z, recurrent_r, recurrent_h = np.split(matrix_inner,3,axis=-1)
    
    z = sigmoid(x_z + recurrent_z)
    r = sigmoid(x_r + recurrent_r)
    hh = tanh(x_h + r * recurrent_h)
    
    h = z * cell_states + (1-z) * hh
    return h

In [11]:
def GlobalAveragePooling1D(x,step_axis=0):
    return np.mean(x,axis=step_axis)

In [15]:
def LayerNormalization(x,gamma,beta,step_axis = -1,epsilon=1e-3):
    mean = np.mean(x,axis = step_axis)
    mean = np.expand_dims(mean,axis=1)
    variance = np.var(x,axis = step_axis)
    variance = np.expand_dims(variance,axis=1)
    inv = 1.0 / np.sqrt(variance + epsilon)
#     print(np.shape(inv))
    gamma = np.expand_dims(gamma,axis=0)
    beta = np.expand_dims(beta,axis=0)
    inv = gamma *inv
    return x * inv + (beta - mean * inv)

In [16]:
def dense(x,gamma, bias):
#     print(np.shape(x))
    y = np.matmul(x,gamma)
    y = np.add(y,bias)
    return y 

In [18]:
def get_weight(file_path):
    f = h5py.File(file_path,'r')
    
    embed = f['embedding']['embedding']['embeddings:0'][:]
    
    forword_gru_bias = f['bidirectional']['bidirectional']['forward_gru']['gru_cell_1']['bias:0'][:]
    forword_gru_kernel = f['bidirectional']['bidirectional']['forward_gru']['gru_cell_1']['kernel:0'][:]
    forword_gru_recurrent_kernel = f['bidirectional']['bidirectional']['forward_gru']['gru_cell_1']['recurrent_kernel:0'][:]
    
    backward_gru_bias = f['bidirectional']['bidirectional']['backward_gru']['gru_cell_2']['bias:0'][:]
    backward_gru_kernel = f['bidirectional']['bidirectional']['backward_gru']['gru_cell_2']['kernel:0'][:]
    backward_gru_recurrent_kernel = f['bidirectional']['bidirectional']['backward_gru']['gru_cell_2']['recurrent_kernel:0'][:]
    
    layer_normal_beta = f['layer_normalization']['layer_normalization']['beta:0'][:]
    layer_normal_gamma = f['layer_normalization']['layer_normalization']['gamma:0'][:]
    
    layer_normal_beta1 = f['layer_normalization_1']['layer_normalization_1']['beta:0'][:]
    layer_normal_gamma1 = f['layer_normalization_1']['layer_normalization_1']['gamma:0'][:]
    
    pre_intent_bias = f['pre_intent']['pre_intent']['bias:0'][:]
    pre_intent_gamma = f['pre_intent']['pre_intent']['kernel:0'][:]
    
    pre_ner_bias = f['pre_ner']['pre_ner']['bias:0'][:]
    pre_ner_gamma = f['pre_ner']['pre_ner']['kernel:0'][:]
    
    return embed, forword_gru_bias,forword_gru_kernel,forword_gru_recurrent_kernel, backward_gru_bias,backward_gru_kernel,backward_gru_recurrent_kernel,\
            layer_normal_beta,layer_normal_gamma,layer_normal_beta1,layer_normal_gamma1,pre_intent_bias,pre_intent_gamma,pre_ner_bias,pre_ner_gamma

In [19]:
def trans2labelid(vocab, labels, max_sent_len):
    labels = [vocab[label] for label in labels]
    if len(labels) < max_sent_len:
        labels += [0] * (max_sent_len - len(labels))
    else:
        labels = labels[:max_sent_len]
    return labels

In [20]:
def test(inputs,h5file_path):
    embed, forword_gru_bias,forword_gru_kernel,forword_gru_recurrent_kernel, backward_gru_bias,backward_gru_kernel,backward_gru_recurrent_kernel,\
            layer_normal_beta,layer_normal_gammma,layer_normal_beta1,layer_normal_gamma1,pre_intent_bias,pre_intent_gamma,pre_ner_bias,pre_ner_gamma = get_weight(h5file_path)
    forword_gru_state = np.zeros((64))
    backed_gru_state = np.zeros((64))
    x = trans2labelid(char2id,inputs,params['max_sent_len'])
    print(x)
    embed = embedding(x,params['embed_size'], embed)
#     print('embed: {}'.format(embed))
    recurrent_embed = embed[::-1]
    forword_gru = []
    backed_gru = []
#     print(np.shape(recurrent_embed))
    for i in range(len(x)):
#         print(np.shape(embed[i]))
        forward_h = GRU(embed[i],forword_gru_state,forword_gru_kernel,forword_gru_recurrent_kernel,forword_gru_bias)
        backed_h =  GRU(recurrent_embed[i], backed_gru_state,backward_gru_kernel,backward_gru_recurrent_kernel,backward_gru_bias)
        forword_gru_state = forward_h
        backed_gru_state = backed_h
        forword_gru.append(forward_h)
        backed_gru.append(backed_h)
    gru_out = np.concatenate((forword_gru,backed_gru[::-1]), axis=-1)
#     print('gru_out: {}'.format(gru_out))
#     print(np.shape(gru_out))
    x_in = LayerNormalization(gru_out,layer_normal_gammma,layer_normal_beta)
    print(np.shape(x_in))
    print('x_in : ',x_in)
    x_conv = GlobalAveragePooling1D(x_in)
    print(np.shape(x_conv))
    pre_intent = dense(x_conv,pre_intent_gamma,pre_intent_bias)
    pre_intent = sigmoid(pre_intent)
    
    x_ner = LayerNormalization(gru_out,layer_normal_gamma1,layer_normal_beta1)
    pre_slot = dense(x_ner,pre_ner_gamma,pre_ner_bias)
    pre_slot = sigmoid(pre_slot)
    
    return embed,gru_out,x_in,x_conv,x_ner,pre_intent,pre_slot


In [21]:
inputs = '打开空调'
h5file_path = '../ner_model_weight/model_conv_625.h5'

In [22]:
embed_np,gru_out_np,x_in_np,x_conv_np,x_ner_np,pre_intent_np,pre_slot_np = test(inputs,h5file_path) 

[111, 196, 182, 95, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(20, 128)
x_in :  [[-0.18487395  2.59982342  0.08869394 ... -0.6315285   0.83122395
   2.1741613 ]
 [ 0.23971886  2.46877678  0.34089185 ...  0.08862706 -1.99671927
   3.54785764]
 [ 0.05058586  0.85279782 -0.05753813 ... -1.72621784  0.79041573
   3.30894318]
 ...
 [ 0.67766895  0.88074969  0.44668887 ... -0.35083664  1.60386768
   0.19373351]
 [ 0.68363193  0.8517546   0.41826727 ... -0.12744785  1.64226132
   0.26700695]
 [ 0.70110929  0.83443363  0.39363473 ...  0.23800833  1.40505977
   0.46103015]]
(128,)


In [23]:
pre_intent_np

array([5.26486299e-11, 3.44658727e-11, 7.77707427e-09, 1.26930422e-11,
       2.39740630e-09, 2.37031140e-08, 2.14346056e-08, 4.69827012e-11,
       1.30352383e-09, 1.77831338e-05, 3.72474467e-18, 9.79177235e-09,
       2.65465432e-08, 3.74631898e-17, 2.47004340e-07, 3.63699069e-16,
       3.01376151e-11, 2.97858980e-11, 5.10297266e-07, 1.29272860e-09,
       5.80643838e-07, 5.29078474e-12, 4.19446210e-11, 3.20406996e-09,
       7.69364845e-08, 5.62570002e-08, 7.43173911e-16, 7.21122280e-15,
       5.39827637e-05, 8.54081832e-12, 2.37959413e-11, 1.57130502e-07,
       1.58208528e-10, 4.44693206e-13, 7.96728843e-11, 9.69827632e-15,
       4.85809455e-10, 2.56163623e-10, 1.71254769e-13, 9.24184487e-12,
       1.18567958e-14, 1.68738497e-09, 7.36440718e-12, 9.47962585e-11,
       5.03368737e-11, 1.62653677e-16, 1.49624083e-11, 4.66498586e-10,
       2.77016688e-12, 5.28493109e-12, 8.74386028e-13, 6.55696446e-11,
       7.77618964e-11, 1.03084342e-11, 3.62505197e-11])

In [24]:
pre_slot_np

array([[3.41983295e-10, 1.98325884e-10, 3.42983686e-13, 3.25632466e-14,
        9.06448294e-15, 1.37813124e-06, 4.37848481e-10, 1.64528917e-11,
        4.15654891e-12, 1.11682936e-11, 3.35575006e-11, 1.86729512e-15,
        3.08152763e-15, 5.01298455e-15, 1.60529039e-11, 1.78041051e-11,
        2.07299795e-16, 8.21172541e-12, 3.94146684e-15, 4.24800101e-10,
        7.05444796e-10, 1.53546333e-09, 1.93005401e-14, 3.92427647e-14,
        3.19460734e-12, 3.59195040e-11, 1.75251329e-12, 4.98750425e-10,
        5.12518727e-11, 4.04607417e-11, 1.62623767e-11, 6.31760309e-12,
        9.99843593e-01, 6.92694309e-12, 1.06972354e-07, 2.61576098e-09],
       [1.81301288e-08, 7.26018990e-11, 5.97766727e-25, 6.18280565e-17,
        1.11609012e-14, 1.94389906e-12, 4.63472514e-15, 5.34867266e-12,
        7.53658391e-16, 3.90022717e-21, 5.96289851e-20, 1.54501844e-10,
        5.73061121e-18, 9.27800754e-17, 6.87489875e-09, 8.02665226e-17,
        1.68309607e-19, 4.05668552e-17, 1.15049796e-19, 1.62079

# 前向与原始模型对比验证

In [25]:
params['intent_num'] = len(intent2id)
params['slot_num'] = len(slot2id)
params['id2intent'] = id2intent
params['id2slot'] = id2slot

In [26]:
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')

tf.keras.backend.clear_session()
text_inputs = tf.keras.layers.Input(shape=(20,),name='Input')
embed = tf.keras.layers.Embedding(500,32)(text_inputs)
bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,return_sequences=True))(embed)
x_in = tf.keras.layers.LayerNormalization()(bilstm)
x_conv = tf.keras.layers.GlobalAveragePooling1D()(x_in)
pre_intent = tf.keras.layers.Dense(params['intent_num'],\
            activation='sigmoid',name = 'pre_intent')(x_conv)
x_ner  = tf.keras.layers.LayerNormalization()(bilstm)
pre_slot = tf.keras.layers.Dense(params['slot_num'],activation='sigmoid',name = 'pre_ner')(x_ner)
model = tf.keras.Model(text_inputs,[pre_intent,pre_slot])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 32)       16000       Input[0][0]                      
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 20, 128)      37632       embedding[0][0]                  
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 20, 128)      256         bidirectional[0][0]              
_______________________________________________________________________________________

In [27]:
model.load_weights('../ner_model_weight/model_conv_625.h5')

In [28]:
x = trans2labelid(char2id,inputs,params['max_sent_len'])

In [29]:
pre_intent,pre_slot = model.predict([x])

In [34]:
pre_intent

array([[5.2648812e-11, 3.4465885e-11, 7.7770652e-09, 1.2693058e-11,
        2.3974007e-09, 2.3703075e-08, 2.1434621e-08, 4.6982664e-11,
        1.3035236e-09, 1.7783097e-05, 3.7247546e-18, 9.7917763e-09,
        2.6546489e-08, 3.7463165e-17, 2.4700412e-07, 3.6369755e-16,
        3.0137576e-11, 2.9785951e-11, 5.1029684e-07, 1.2927308e-09,
        5.8064313e-07, 5.2907899e-12, 4.1944562e-11, 3.2040588e-09,
        7.6936509e-08, 5.6257043e-08, 7.4317374e-16, 7.2112014e-15,
        5.3982822e-05, 8.5408043e-12, 2.3795993e-11, 1.5713033e-07,
        1.5820807e-10, 4.4469354e-13, 7.9673032e-11, 9.6982918e-15,
        4.8580800e-10, 2.5616401e-10, 1.7125476e-13, 9.2418590e-12,
        1.1856786e-14, 1.6873805e-09, 7.3644025e-12, 9.4796331e-11,
        5.0336926e-11, 1.6265312e-16, 1.4962413e-11, 4.6649712e-10,
        2.7701658e-12, 5.2849400e-12, 8.7439057e-13, 6.5569758e-11,
        7.7761679e-11, 1.0308426e-11, 3.6250520e-11]], dtype=float32)

In [35]:
pre_slot

array([[[3.41982748e-10, 1.98325759e-10, 3.42984294e-13, 3.25633245e-14,
         9.06449174e-15, 1.37813151e-06, 4.37848285e-10, 1.64528790e-11,
         4.15655227e-12, 1.11683050e-11, 3.35575352e-11, 1.86729112e-15,
         3.08153297e-15, 5.01299636e-15, 1.60529090e-11, 1.78041974e-11,
         2.07299786e-16, 8.21171516e-12, 3.94146114e-15, 4.24800695e-10,
         7.05447090e-10, 1.53546631e-09, 1.93005300e-14, 3.92427756e-14,
         3.19460756e-12, 3.59195347e-11, 1.75251849e-12, 4.98752539e-10,
         5.12518604e-11, 4.04608187e-11, 1.62623751e-11, 6.31762481e-12,
         9.99843597e-01, 6.92695684e-12, 1.06972749e-07, 2.61576516e-09],
        [1.81301196e-08, 7.26020910e-11, 5.97769508e-25, 6.18278465e-17,
         1.11609203e-14, 1.94389838e-12, 4.63470764e-15, 5.34866552e-12,
         7.53659953e-16, 3.90024343e-21, 5.96290658e-20, 1.54501925e-10,
         5.73060637e-18, 9.27806009e-17, 6.87492285e-09, 8.02664369e-17,
         1.68309683e-19, 4.05668728e-17, 1.1505032